# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “Shaping data with tuples”

>Tuples are nice in that they allow you to pass arbitrary bits of data around, temporarily grouped together. Tuples also support equality comparison by default, so you can compare arbitrary tuples against one another (provided their generic types are the same, and each type itself supports equality comparison).

In 2017, C# 7.0 was released with the newer `System.ValueTuple` [📖 [docs](https://docs.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-7#tuples-and-discards)] compared to the `System.Tuple` recognized by Isaac Abraham (c. 2018). This means the `System.Tuple` example in the book can be replaced by the C# 7.0 equivalent:


In [ ]:
using System;
using System.Linq;

public (string forename, string surname) ParseName(string name)
{
    if(string.IsNullOrWhiteSpace(name)) throw new ArgumentNullException(nameof(name));

    string[] parts = name.Split(' ');

    return (parts.First(), parts.Last());
}

var (forename, surname) = ParseName("Isaac Abraham");

$"{nameof(forename)}: {forename}; {nameof(surname)}: {surname}"

forename: Isaac; surname: Abraham

The F# equivalent:


In [ ]:
#!fsharp

open System
open System.Linq

let parseName(name) =
    if(String.IsNullOrWhiteSpace name) then nullArg(nameof name)

    let parts = name.Split ' '
    parts.First(), parts.Last()

let forename, surname = parseName "Isaac Abraham"

$"{nameof(forename)}: {forename}; {nameof(surname)}: {surname}"

forename: Isaac; surname: Abraham

The use of `nullArg` [📖 [docs](https://docs.microsoft.com/en-us/dotnet/fsharp/style-guide/conventions#use-exceptions-when-errors-cannot-be-represented-with-types)] above is drifting away from the text a bit but this exception-throwing branch of the `null` check must be included for the sake of equivalency. And this leads to the larger question around the use of `string` in F#: can the use of `string` ‘pollute’ F# with `null` checks?

The answer to the above question is _yes_ and there are at least two ways this can happen:

1. passing `string` into F# from a language that supports `null` (like C#)
2. deliberately and explicitly binding `string` to `null` (see below)


In [ ]:
#!fsharp

let s : string = null

For more depth, see “[Type safety of Option vs. null](https://fsharpforfunandprofit.com/posts/the-option-type/#type-safety-of-option-vs-null).”


## Tuple helpers

>F# also has two built-in functions for working with two-part tuples: `fst` and `snd`.


In [ ]:
#!fsharp

let parsedName = parseName "Isaac Abraham"

fst parsedName

Isaac

In [ ]:
#!fsharp

snd parsedName

Abraham

## Tuple type signatures

>It’s worth understanding tuple notation in F#, which is `type * type * type`.

We can dip into the `FSharp.Reflection` namespace to quickly visualize tuple type signatures with `FSharpValue.GetTupleFields` [📖 [docs](https://fsharp.github.io/fsharp-core-docs/reference/fsharp-reflection-fsharpvalue.html#GetTupleFields)]:


In [ ]:
#!fsharp

open FSharp.Reflection

let nameAndAge = "Joe", "Bloggs", 28

FSharpValue.GetTupleFields nameAndAge

index,type,value
0,System.String,Joe
1,System.String,Bloggs
2,System.Int32,28


## Nested tuples

Use brackets (parentheses) to “nest” tuples:


In [ ]:
#!fsharp

let nestedNameAndAge = ("Isaac", "Abraham"), 28

FSharpValue.GetTupleFields nestedNameAndAge

index,type,Item1,Item2,value
0,"System.Tuple<System.String,System.String>",Isaac,Abraham,
1,System.Int32,,,28


We can deconstruct into bindings on the nested tuple level:


In [ ]:
#!fsharp

let name, age = nestedNameAndAge

FSharpValue.GetTupleFields name

index,value
0,Isaac
1,Abraham


In [ ]:
#!fsharp

age

28

We can deconstruct into bindings on the scalar level:


In [ ]:
#!fsharp

let (firstName, lastName), yearsOld = nestedNameAndAge

$@"
{nameof(firstName)}: {firstName}
{nameof(lastName)}: {lastName}
{nameof(yearsOld)}: {yearsOld}
"


firstName: Isaac
lastName: Abraham
yearsOld: 28


## Wildcards (discards)

>If there are elements of a tuple you are not interested in, you can discard them while deconstructing a tuple by assigning those parts to the underscore symbol.


In [ ]:
#!fsharp

let (fName, lName), _ = nestedNameAndAge

$@"
{nameof(fName)}: {fName}
{nameof(lName)}: {lName}
"


fName: Isaac
lName: Abraham


We can use this technique on the tuple level and test equality between tuples:


In [ ]:
#!fsharp

let nestedName, _ = nestedNameAndAge

parsedName = nestedName

True

C# 7.0 introduced the use of this syntax with `_`, calling them _discards_ [📖 [docs](https://docs.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-7#tuples-and-discards)].


[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.252001+662171b00f676a06b8db90dd51fa910aecbcd492Build date: 2021-10-21T10:23:34.0000000Zhttps://github.com/dotnet/interactive
